In [53]:
## experimenting with the top 68 features those had come from feature importance analysis

In [21]:
nl = ['ExterQual',
 'GarageCars',
 'KitchenQual',
 'BsmtQual',
 'OverallQual',
 'Neighborhood',
 'FullBath',
 'GrLivArea',
 'FireplaceQu',
 '2ndFlrSF',
 'GarageArea',
 'TotRmsAbvGrd',
 'LowQualFinSF',
 'TotalBsmtSF',
 'PoolArea',
 'MSSubClass',
 '1stFlrSF',
 'BldgType',
 'Fireplaces',
 'CentralAir',
 'GarageFinish',
 'MasVnrType',
 'LandSlope',
 'BsmtFinSF1',
 'ScreenPorch',
 'BedroomAbvGr',
 'BsmtFullBath',
 'BsmtFinType1',
 'LandContour',
 'YearBuilt',
 'HalfBath',
 'PoolQC',
 'BsmtExposure',
 'Age_of_house',
 'LotArea',
 'GarageQual',
 'years_since_remodelled',
 'LotShape',
 'Foundation',
 'Condition1',
 'GarageType',
 'KitchenAbvGr',
 'Exterior1st',
 'SaleType',
 'MSZoning',
 'HouseStyle',
 'GarageYrBlt',
 'WoodDeckSF',
 'YearRemodAdd',
 'OpenPorchSF',
 'ExterCond',
 'Exterior2nd',
 'RoofStyle',
 '3SsnPorch',
 'MasVnrArea',
 'BsmtUnfSF',
 'OverallCond',
 'MiscVal',
 'Functional',
 'SaleCondition',
 'BsmtCond',
 'EnclosedPorch',
 'LotFrontage',
 'RoofMatl',
 'HeatingQC',
 'MoSold',
 'BsmtFinSF2',
 'Heating']

In [54]:
import pyforest
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder ,StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import flaml
from catboost import CatBoostRegressor
import mlflow
from category_encoders import TargetEncoder
from autofeat import AutoFeatRegressor
import math
from flaml import AutoML

In [55]:
df  = pd.read_csv('/Users/karanam.rama/Downloads/house_price_pred_repo/datas/raw/house_pred_raw_data.csv')
df['MSSubClass'] =df['MSSubClass'].astype('str')
# df['price_per_sqft'] = df['SalePrice']/df['LotArea']
## removing the houses with prices greater than 500000
# df = df[df['SalePrice']<500000]
print('shape of the dataframe',df.shape)
print('data frame head')
df.head()

<IPython.core.display.Javascript object>

shape of the dataframe (1460, 81)
data frame head


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [56]:
df['Fence']=df['Fence'].fillna('NA')
df['PoolQC'] = df['PoolQC'].fillna('NA')
df['MiscFeature'] = df['MiscFeature'].fillna('NA')
df['Alley'] = df['Alley'].fillna('NA')
df['Fence'] = df['Fence'].fillna('NA')
df['FireplaceQu'] = df['FireplaceQu'].fillna('NA')
df['LotFrontage'] = df['LotFrontage'].fillna(0)

ndf = pd.DataFrame(df.isnull().mean())
ndf.columns = ['null_mean']
ndf = ndf[ndf['null_mean']>0]
ndf['column_name'] = ndf.index


num_list = df[ndf['column_name']].select_dtypes(include=['number']).columns
obj_list = df[ndf['column_name']].select_dtypes(exclude=['number']).columns

# replacing null of int type columns with their mean values
df[num_list] = SimpleImputer(strategy='mean').fit_transform(df[num_list])

# replacing null of object type columns with the most frequent category
df[obj_list] = SimpleImputer(strategy='most_frequent').fit_transform(df[obj_list])

df.set_index(['Id'],inplace = True)

x = df.drop('SalePrice',axis = 1)
y = df['SalePrice']
all_columns = x.columns
cat_columns = x.select_dtypes(include=['object']).columns
num_columns = x.select_dtypes(exclude=['object']).columns

# creating a new data frame with numerical columns only
num_x = x[num_columns]
num_x['Age_of_house'] = num_x['YrSold'] - num_x['YearBuilt']
num_x['years_since_remodelled'] = num_x['YrSold'] - num_x['YearRemodAdd']

# preproccessing the numerical data of x using standard scaler and minmax scaler
ss = StandardScaler()
mms = MinMaxScaler()
num_x_ss = pd.DataFrame(ss.fit_transform(num_x),columns=num_x.columns,index = num_x.index)
num_x_ss_mms  =  pd.DataFrame(mms.fit_transform(num_x_ss),columns=num_x.columns,index = num_x.index)

#preproccessing the categorical data of x using Target Encoder ,standard scaler and minmax scaler
cat_x =  TargetEncoder().fit_transform(x[cat_columns],y)
cat_x_ss = pd.DataFrame(ss.fit_transform(cat_x),columns=cat_x.columns,index = cat_x.index)
cat_x_ss_mms  =  pd.DataFrame(mms.fit_transform(cat_x_ss),columns=cat_x.columns,index = cat_x.index)


# joining the both dataframes 

final_x = num_x_ss_mms.join(cat_x_ss_mms,how='inner')

<IPython.core.display.Javascript object>

<ipython-input-56-243b0d67125e>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_x['Age_of_house'] = num_x['YrSold'] - num_x['YearBuilt']
<ipython-input-56-243b0d67125e>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_x['years_since_remodelled'] = num_x['YrSold'] - num_x['YearRemodAdd']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [57]:
x_train,x_test,y_train,y_test = train_test_split(final_x,y,random_state=42,test_size=0.3)
# x_train = x_train[nl]
# x_test = x_test[nl]

In [58]:
xgbr = XGBRegressor()
xgbr.fit(x_train,y_train)
y_pred = xgbr.predict(x_test)
rmse = mean_squared_error(y_test,y_pred,squared=False)
mse = mean_squared_error(y_test,y_pred,squared=True)
mae = mean_absolute_error(y_test,y_pred)
explained_variance_score = sklearn.metrics.explained_variance_score(y_test,y_pred)
r2 = sklearn.metrics.r2_score(y_test,y_pred)
print('rmse ', rmse)
print('mse ', mse)
print('mae ', mae)
print('explained_variance_score',explained_variance_score)
print('r2',r2)
# 0.9657

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

rmse  25532.854189660542
mse  651926643.0704659
mae  16343.458181720891
explained_variance_score 0.9066353069966839
r2 0.9065752531558545


In [59]:
## for training data
xgbr = XGBRegressor()
xgbr.fit(x_train,y_train)
y_pred = xgbr.predict(x_train)
rmse = mean_squared_error(y_train,y_pred,squared=False)
mse = mean_squared_error(y_train,y_pred,squared=True)
mae = mean_absolute_error(y_train,y_pred)
explained_variance_score = sklearn.metrics.explained_variance_score(y_train,y_pred)
r2 = sklearn.metrics.r2_score(y_train,y_pred)
print('rmse ', rmse)
print('mse ', mse)
print('mae ', mae)
print('explained_variance_score',explained_variance_score)
print('r2',r2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

rmse  758.5730446754746
mse  575433.0641082196
mae  515.11627400318
explained_variance_score 0.9999043906000309
r2 0.999904390487886


In [29]:
lgbmr = LGBMRegressor()
lgbmr.fit(x_train,y_train)
y_pred = lgbmr.predict(x_test)
rmse = mean_squared_error(y_test,y_pred,squared=False)
mse = mean_squared_error(y_test,y_pred,squared=True)
mae = mean_absolute_error(y_test,y_pred)
explained_variance_score = sklearn.metrics.explained_variance_score(y_test,y_pred)
r2 = sklearn.metrics.r2_score(y_test,y_pred)
print('rmse ', rmse)
print('mse ', mse)
print('mae ', mae)
print('explained_variance_score',explained_variance_score)
print('r2',r2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

rmse  25904.933040739073
mse  671065555.845175
mae  15787.046104229308
explained_variance_score 0.9038963906406858
r2 0.9038325395394452


In [30]:
## for training data
lgbmr = LGBMRegressor()
lgbmr.fit(x_train,y_train)
y_pred = lgbmr.predict(x_train)
rmse = mean_squared_error(y_train,y_pred,squared=False)
mse = mean_squared_error(y_train,y_pred,squared=True)
mae = mean_absolute_error(y_train,y_pred)
explained_variance_score = sklearn.metrics.explained_variance_score(y_train,y_pred)
r2 = sklearn.metrics.r2_score(y_train,y_pred)
print('rmse ', rmse)
print('mse ', mse)
print('mae ', mae)
print('explained_variance_score',explained_variance_score)
print('r2',r2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

rmse  10945.726131103444
mse  119808920.53712077
mae  4946.017083425286
explained_variance_score 0.9800934754119323
r2 0.9800934754119323


In [31]:
cbr = CatBoostRegressor()
cbr.fit(x_train,y_train)
y_pred = lgbmr.predict(x_test)

Learning rate set to 0.039182
0:	learn: 75843.4285764	total: 3.55ms	remaining: 3.55s
1:	learn: 73998.2869606	total: 5.15ms	remaining: 2.57s
2:	learn: 72221.2350634	total: 6.92ms	remaining: 2.3s
3:	learn: 70491.8671449	total: 8.58ms	remaining: 2.13s
4:	learn: 68982.3042215	total: 10.1ms	remaining: 2.01s
5:	learn: 67500.2468368	total: 11.9ms	remaining: 1.97s
6:	learn: 66038.3520860	total: 13.6ms	remaining: 1.93s
7:	learn: 64585.3771393	total: 15.4ms	remaining: 1.91s
8:	learn: 63347.4725298	total: 17.2ms	remaining: 1.89s
9:	learn: 61890.9938556	total: 18.7ms	remaining: 1.85s
10:	learn: 60724.9704785	total: 20.3ms	remaining: 1.83s
11:	learn: 59500.2982494	total: 22.1ms	remaining: 1.82s
12:	learn: 58254.0012401	total: 24ms	remaining: 1.82s
13:	learn: 57012.5017240	total: 25.5ms	remaining: 1.8s
14:	learn: 55962.8791878	total: 27.2ms	remaining: 1.79s
15:	learn: 54875.9135049	total: 28.8ms	remaining: 1.77s
16:	learn: 53736.3436881	total: 30.3ms	remaining: 1.75s
17:	learn: 52643.8391469	total: 

248:	learn: 15322.8275721	total: 341ms	remaining: 1.03s
249:	learn: 15304.2697509	total: 343ms	remaining: 1.03s
250:	learn: 15279.8039483	total: 344ms	remaining: 1.03s
251:	learn: 15241.3300752	total: 345ms	remaining: 1.02s
252:	learn: 15200.4484084	total: 347ms	remaining: 1.02s
253:	learn: 15173.7690091	total: 348ms	remaining: 1.02s
254:	learn: 15138.5526294	total: 349ms	remaining: 1.02s
255:	learn: 15094.4624903	total: 350ms	remaining: 1.02s
256:	learn: 15056.2042643	total: 352ms	remaining: 1.02s
257:	learn: 15050.1495313	total: 352ms	remaining: 1.01s
258:	learn: 15017.2189336	total: 354ms	remaining: 1.01s
259:	learn: 15011.5272650	total: 355ms	remaining: 1.01s
260:	learn: 14984.7300846	total: 356ms	remaining: 1.01s
261:	learn: 14957.1092346	total: 358ms	remaining: 1.01s
262:	learn: 14931.3810398	total: 359ms	remaining: 1.01s
263:	learn: 14900.9466698	total: 361ms	remaining: 1s
264:	learn: 14862.0679127	total: 362ms	remaining: 1s
265:	learn: 14833.5815176	total: 364ms	remaining: 1s
2

510:	learn: 9667.8204593	total: 700ms	remaining: 669ms
511:	learn: 9656.0251239	total: 701ms	remaining: 668ms
512:	learn: 9643.4019750	total: 703ms	remaining: 667ms
513:	learn: 9629.7753843	total: 704ms	remaining: 666ms
514:	learn: 9622.7864189	total: 705ms	remaining: 664ms
515:	learn: 9606.9969529	total: 707ms	remaining: 663ms
516:	learn: 9605.1847855	total: 708ms	remaining: 662ms
517:	learn: 9580.6228895	total: 710ms	remaining: 660ms
518:	learn: 9569.8688904	total: 711ms	remaining: 659ms
519:	learn: 9552.1201326	total: 712ms	remaining: 657ms
520:	learn: 9532.2592600	total: 714ms	remaining: 656ms
521:	learn: 9517.3047238	total: 715ms	remaining: 655ms
522:	learn: 9507.5068576	total: 717ms	remaining: 654ms
523:	learn: 9486.4657634	total: 718ms	remaining: 652ms
524:	learn: 9466.6432423	total: 719ms	remaining: 651ms
525:	learn: 9454.8672263	total: 721ms	remaining: 649ms
526:	learn: 9434.9393705	total: 723ms	remaining: 649ms
527:	learn: 9421.8470043	total: 724ms	remaining: 647ms
528:	learn

783:	learn: 6814.3967956	total: 1.06s	remaining: 292ms
784:	learn: 6813.7274668	total: 1.06s	remaining: 291ms
785:	learn: 6809.8781259	total: 1.06s	remaining: 289ms
786:	learn: 6804.8977114	total: 1.06s	remaining: 288ms
787:	learn: 6800.2029075	total: 1.06s	remaining: 286ms
788:	learn: 6793.3041361	total: 1.07s	remaining: 285ms
789:	learn: 6792.5601517	total: 1.07s	remaining: 284ms
790:	learn: 6790.6590948	total: 1.07s	remaining: 282ms
791:	learn: 6776.2187000	total: 1.07s	remaining: 281ms
792:	learn: 6762.4614206	total: 1.07s	remaining: 280ms
793:	learn: 6752.9150605	total: 1.07s	remaining: 278ms
794:	learn: 6749.6290706	total: 1.07s	remaining: 277ms
795:	learn: 6745.6729847	total: 1.07s	remaining: 276ms
796:	learn: 6739.4965019	total: 1.08s	remaining: 274ms
797:	learn: 6730.2002430	total: 1.08s	remaining: 273ms
798:	learn: 6716.7693634	total: 1.08s	remaining: 272ms
799:	learn: 6714.8748880	total: 1.08s	remaining: 270ms
800:	learn: 6708.2443526	total: 1.08s	remaining: 269ms
801:	learn

In [32]:
rmse = mean_squared_error(y_test,y_pred,squared=False)
mse = mean_squared_error(y_test,y_pred,squared=True)
mae = mean_absolute_error(y_test,y_pred)
explained_variance_score = sklearn.metrics.explained_variance_score(y_test,y_pred)
r2 = sklearn.metrics.r2_score(y_test,y_pred)
print('rmse ', rmse)
print('mse ', mse)
print('mae ', mae)
print('explained_variance_score',explained_variance_score)
print('r2',r2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

rmse  25904.933040739073
mse  671065555.845175
mae  15787.046104229308
explained_variance_score 0.9038963906406858
r2 0.9038325395394452


In [47]:
automl = AutoML()
automl_settings = {"time_budget" : 100,"metric" :  'r2', "task":'regression',"eval_method" : 'cv'}
automl.fit(x_train,y_train,**automl_settings) 

[flaml.automl: 08-19 22:37:42] {1102} INFO - Evaluation method: cv
[flaml.automl: 08-19 22:37:42] {630} INFO - Using RepeatedKFold
[flaml.automl: 08-19 22:37:42] {1123} INFO - Minimizing error metric: 1-r2
[flaml.automl: 08-19 22:37:42] {1143} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree']
[flaml.automl: 08-19 22:37:42] {1210} INFO - iteration 0, current learner lgbm
[flaml.automl: 08-19 22:37:42] {1361} INFO -  at 0.2s,	best lgbm's error=0.6526,	best lgbm's error=0.6526
[flaml.automl: 08-19 22:37:42] {1210} INFO - iteration 1, current learner lgbm
[flaml.automl: 08-19 22:37:42] {1361} INFO -  at 0.3s,	best lgbm's error=0.6526,	best lgbm's error=0.6526
[flaml.automl: 08-19 22:37:42] {1210} INFO - iteration 2, current learner lgbm
[flaml.automl: 08-19 22:37:42] {1361} INFO -  at 0.4s,	best lgbm's error=0.3709,	best lgbm's error=0.3709


Finished loading model, total used 4 iterations
Finished loading model, total used 4 iterations
Finished loading model, total used 4 iterations


[flaml.automl: 08-19 22:37:42] {1210} INFO - iteration 3, current learner xgboost
[flaml.automl: 08-19 22:37:42] {1361} INFO -  at 0.5s,	best xgboost's error=3.0366,	best lgbm's error=0.3709
[flaml.automl: 08-19 22:37:42] {1210} INFO - iteration 4, current learner lgbm
[flaml.automl: 08-19 22:37:42] {1361} INFO -  at 0.6s,	best lgbm's error=0.2952,	best lgbm's error=0.2952
[flaml.automl: 08-19 22:37:42] {1210} INFO - iteration 5, current learner lgbm
[flaml.automl: 08-19 22:37:42] {1361} INFO -  at 0.6s,	best lgbm's error=0.2952,	best lgbm's error=0.2952
[flaml.automl: 08-19 22:37:42] {1210} INFO - iteration 6, current learner lgbm
[flaml.automl: 08-19 22:37:42] {1361} INFO -  at 0.8s,	best lgbm's error=0.2512,	best lgbm's error=0.2512
[flaml.automl: 08-19 22:37:42] {1210} INFO - iteration 7, current learner lgbm


Finished loading model, total used 4 iterations
Finished loading model, total used 4 iterations
Finished loading model, total used 4 iterations


[flaml.automl: 08-19 22:37:42] {1361} INFO -  at 0.9s,	best lgbm's error=0.2455,	best lgbm's error=0.2455
[flaml.automl: 08-19 22:37:42] {1210} INFO - iteration 8, current learner lgbm
[flaml.automl: 08-19 22:37:43] {1361} INFO -  at 1.1s,	best lgbm's error=0.2309,	best lgbm's error=0.2309
[flaml.automl: 08-19 22:37:43] {1210} INFO - iteration 9, current learner lgbm


...

[flaml.automl: 08-19 22:39:18] {1361} INFO -  at 96.8s,	best xgboost's error=0.1387,	best catboost's error=0.1307
[flaml.automl: 08-19 22:39:18] {1210} INFO - iteration 188, current learner lgbm
[flaml.automl: 08-19 22:39:19] {1361} INFO -  at 97.5s,	best lgbm's error=0.1392,	best catboost's error=0.1307
[flaml.automl: 08-19 22:39:19] {1210} INFO - iteration 189, current learner lgbm


Finished loading model, total used 25 iterations


[flaml.automl: 08-19 22:39:21] {1361} INFO -  at 99.0s,	best lgbm's error=0.1392,	best catboost's error=0.1307
[flaml.automl: 08-19 22:39:21] {1210} INFO - iteration 190, current learner extra_tree


Finished loading model, total used 36 iterations


[flaml.automl: 08-19 22:39:21] {1361} INFO -  at 99.9s,	best extra_tree's error=0.1390,	best catboost's error=0.1307
[flaml.automl: 08-19 22:39:21] {1407} INFO - selected model: <catboost.core.CatBoostRegressor object at 0x7f9344f23c40>
[flaml.automl: 08-19 22:39:21] {1161} INFO - fit succeeded


In [48]:
y_pred = automl.predict(x_test)
rmse = mean_squared_error(y_test,y_pred,squared=False)
mse = mean_squared_error(y_test,y_pred,squared=True)
mae = mean_absolute_error(y_test,y_pred)
explained_variance_score = sklearn.metrics.explained_variance_score(y_test,y_pred)
r2 = sklearn.metrics.r2_score(y_test,y_pred)
print('rmse ', rmse)
print('mse ', mse)
print('mae ', mae)
print('explained_variance_score',explained_variance_score)
print('r2',r2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

rmse  26167.953682726722
mse  684761799.941331
mae  15606.587652511631
explained_variance_score 0.901869790894479
r2 0.9018697908912653


In [49]:
## training data
y_pred = automl.predict(x_train)
rmse = mean_squared_error(y_train,y_pred,squared=False)
mse = mean_squared_error(y_train,y_pred,squared=True)
mae = mean_absolute_error(y_train,y_pred)
explained_variance_score = sklearn.metrics.explained_variance_score(y_train,y_pred)
r2 = sklearn.metrics.r2_score(y_train,y_pred)
print('rmse ', rmse)
print('mse ', mse)
print('mae ', mae)
print('explained_variance_score',explained_variance_score)
print('r2',r2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

rmse  16550.417061697903
mse  273916304.9161411
mae  11537.327329835094
explained_variance_score 0.9544884597906392
r2 0.9544881830631603


In [50]:
automl.best_config

{'early_stopping_rounds': 11, 'learning_rate': 0.06392848105163096}

In [51]:
automl.model.estimator

In [52]:
cbr = CatBoostRegressor(**automl.best_config)
cbr.fit(x_train,y_train)
y_pred = cbr.predict(x_test)
rmse = mean_squared_error(y_test,y_pred,squared=False)
mse = mean_squared_error(y_test,y_pred,squared=True)
mae = mean_absolute_error(y_test,y_pred)
explained_variance_score = sklearn.metrics.explained_variance_score(y_test,y_pred)
r2 = sklearn.metrics.r2_score(y_test,y_pred)
print('rmse ', rmse)
print('mse ', mse)
print('mae ', mae)
print('explained_variance_score',explained_variance_score)
print('r2',r2)

0:	learn: 74776.7693496	total: 3.92ms	remaining: 3.92s
1:	learn: 72048.8321485	total: 6.19ms	remaining: 3.09s
2:	learn: 69354.4897900	total: 8.19ms	remaining: 2.72s
3:	learn: 66669.1962429	total: 10.7ms	remaining: 2.67s
4:	learn: 64280.0284212	total: 13ms	remaining: 2.59s
5:	learn: 62101.1913758	total: 15.3ms	remaining: 2.53s
6:	learn: 59915.4683188	total: 17.8ms	remaining: 2.52s
7:	learn: 57786.8260287	total: 20.2ms	remaining: 2.5s
8:	learn: 55800.0055501	total: 22.4ms	remaining: 2.47s
9:	learn: 53971.3490580	total: 24.8ms	remaining: 2.45s
10:	learn: 52260.1516110	total: 26.8ms	remaining: 2.41s
11:	learn: 50649.3573709	total: 28.7ms	remaining: 2.36s
12:	learn: 49200.3686219	total: 31.2ms	remaining: 2.37s
13:	learn: 47874.7294675	total: 33.4ms	remaining: 2.35s
14:	learn: 46338.1019239	total: 35.4ms	remaining: 2.33s
15:	learn: 44982.1233127	total: 37.6ms	remaining: 2.31s
16:	learn: 43807.0812288	total: 39.7ms	remaining: 2.29s
17:	learn: 42645.1305053	total: 41.8ms	remaining: 2.28s
18:	l

185:	learn: 13829.1348151	total: 371ms	remaining: 1.62s
186:	learn: 13773.7311669	total: 373ms	remaining: 1.62s
187:	learn: 13729.0290893	total: 375ms	remaining: 1.62s
188:	learn: 13710.2545620	total: 377ms	remaining: 1.62s
189:	learn: 13664.6635715	total: 379ms	remaining: 1.61s
190:	learn: 13619.9345472	total: 381ms	remaining: 1.61s
191:	learn: 13551.0851732	total: 383ms	remaining: 1.61s
192:	learn: 13514.4316540	total: 385ms	remaining: 1.61s
193:	learn: 13471.3841891	total: 387ms	remaining: 1.6s
194:	learn: 13424.0971209	total: 388ms	remaining: 1.6s
195:	learn: 13363.7538274	total: 390ms	remaining: 1.6s
196:	learn: 13289.7826530	total: 392ms	remaining: 1.6s
197:	learn: 13252.8173770	total: 394ms	remaining: 1.6s
198:	learn: 13198.7557636	total: 397ms	remaining: 1.6s
199:	learn: 13138.7714156	total: 399ms	remaining: 1.6s
200:	learn: 13072.1383283	total: 401ms	remaining: 1.59s
201:	learn: 13029.8269236	total: 403ms	remaining: 1.59s
202:	learn: 12997.0190259	total: 406ms	remaining: 1.59s

366:	learn: 8589.4600508	total: 737ms	remaining: 1.27s
367:	learn: 8567.0211802	total: 740ms	remaining: 1.27s
368:	learn: 8551.7152844	total: 742ms	remaining: 1.27s
369:	learn: 8544.4591481	total: 744ms	remaining: 1.27s
370:	learn: 8526.7340829	total: 746ms	remaining: 1.26s
371:	learn: 8486.0793243	total: 748ms	remaining: 1.26s
372:	learn: 8480.4532195	total: 750ms	remaining: 1.26s
373:	learn: 8459.2316392	total: 752ms	remaining: 1.26s
374:	learn: 8427.6494852	total: 755ms	remaining: 1.26s
375:	learn: 8394.9869672	total: 757ms	remaining: 1.25s
376:	learn: 8391.2685732	total: 759ms	remaining: 1.25s
377:	learn: 8370.4416848	total: 761ms	remaining: 1.25s
378:	learn: 8344.0517364	total: 763ms	remaining: 1.25s
379:	learn: 8320.4495927	total: 766ms	remaining: 1.25s
380:	learn: 8304.3311473	total: 768ms	remaining: 1.25s
381:	learn: 8283.1194840	total: 770ms	remaining: 1.25s
382:	learn: 8262.0121177	total: 772ms	remaining: 1.24s
383:	learn: 8254.9820926	total: 774ms	remaining: 1.24s
384:	learn

539:	learn: 6166.5682864	total: 1.1s	remaining: 938ms
540:	learn: 6151.3106871	total: 1.1s	remaining: 937ms
541:	learn: 6135.5922542	total: 1.11s	remaining: 935ms
542:	learn: 6126.6149976	total: 1.11s	remaining: 933ms
543:	learn: 6108.5876007	total: 1.11s	remaining: 931ms
544:	learn: 6093.2042269	total: 1.11s	remaining: 929ms
545:	learn: 6092.3137688	total: 1.11s	remaining: 927ms
546:	learn: 6072.8232339	total: 1.12s	remaining: 925ms
547:	learn: 6068.3069676	total: 1.12s	remaining: 923ms
548:	learn: 6041.1617851	total: 1.12s	remaining: 921ms
549:	learn: 6026.5764099	total: 1.12s	remaining: 918ms
550:	learn: 6012.1315709	total: 1.12s	remaining: 917ms
551:	learn: 6006.9971698	total: 1.13s	remaining: 915ms
552:	learn: 5992.9698641	total: 1.13s	remaining: 913ms
553:	learn: 5981.4149713	total: 1.13s	remaining: 911ms
554:	learn: 5980.6150407	total: 1.13s	remaining: 909ms
555:	learn: 5971.5609609	total: 1.14s	remaining: 907ms
556:	learn: 5953.0709969	total: 1.14s	remaining: 905ms
557:	learn: 

724:	learn: 4420.5143249	total: 1.47s	remaining: 557ms
725:	learn: 4411.2561115	total: 1.47s	remaining: 555ms
726:	learn: 4406.1364835	total: 1.47s	remaining: 553ms
727:	learn: 4394.5931702	total: 1.47s	remaining: 551ms
728:	learn: 4384.9854292	total: 1.48s	remaining: 549ms
729:	learn: 4381.1314609	total: 1.48s	remaining: 547ms
730:	learn: 4375.2003096	total: 1.48s	remaining: 545ms
731:	learn: 4366.8764252	total: 1.48s	remaining: 543ms
732:	learn: 4356.8417315	total: 1.48s	remaining: 541ms
733:	learn: 4354.7101391	total: 1.49s	remaining: 539ms
734:	learn: 4344.8709893	total: 1.49s	remaining: 537ms
735:	learn: 4331.3560639	total: 1.49s	remaining: 535ms
736:	learn: 4326.9120503	total: 1.49s	remaining: 533ms
737:	learn: 4324.2024406	total: 1.49s	remaining: 531ms
738:	learn: 4321.8347546	total: 1.5s	remaining: 529ms
739:	learn: 4318.4051820	total: 1.5s	remaining: 527ms
740:	learn: 4315.4056295	total: 1.5s	remaining: 525ms
741:	learn: 4313.2055275	total: 1.5s	remaining: 522ms
742:	learn: 43

899:	learn: 3326.3424741	total: 1.83s	remaining: 204ms
900:	learn: 3320.5472279	total: 1.83s	remaining: 202ms
901:	learn: 3314.9949709	total: 1.84s	remaining: 200ms
902:	learn: 3307.4223468	total: 1.84s	remaining: 198ms
903:	learn: 3302.8849986	total: 1.84s	remaining: 196ms
904:	learn: 3300.0248930	total: 1.84s	remaining: 194ms
905:	learn: 3293.0106077	total: 1.84s	remaining: 191ms
906:	learn: 3290.5340041	total: 1.85s	remaining: 189ms
907:	learn: 3284.5909922	total: 1.85s	remaining: 187ms
908:	learn: 3279.6552222	total: 1.85s	remaining: 185ms
909:	learn: 3274.1201950	total: 1.85s	remaining: 183ms
910:	learn: 3271.0138694	total: 1.86s	remaining: 181ms
911:	learn: 3266.0022047	total: 1.86s	remaining: 179ms
912:	learn: 3263.5097352	total: 1.86s	remaining: 177ms
913:	learn: 3259.2650064	total: 1.86s	remaining: 175ms
914:	learn: 3255.8268291	total: 1.86s	remaining: 173ms
915:	learn: 3248.8044141	total: 1.87s	remaining: 171ms
916:	learn: 3242.7411473	total: 1.87s	remaining: 169ms
917:	learn

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

rmse  23989.067318289435
mse  575475350.8014222
mae  14287.449254509771
explained_variance_score 0.9175312147033817
r2 0.9175311524154759
